## Junos PyEZ RPC Execution
Junos uses XML natively (Extensible Markup Language) modelling. While the CLI is optimized for humans, XML is not optimized for humans as it has too much markup. However, XML can be manipulated by programs. When a human interacts with the Junos CLI, it passes the equivalent XML RPC to MGD (management daemon). The MGD returns the data response to the CLI in the form of an XML document and CLI converts into a human readable format for display.

Every CLI command has an equivalent XML RPC associated with it. To display the output of a Junos CLI command in XML format, append “| display xml” to the CLI command. Likewise appending “| display xml rpc” option provides the RPC to get an XML encoded response. 

    seanw@vsrx1> show version | display xml rpc 
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <rpc>
            <get-software-information>
            </get-software-information>
        </rpc>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>
    
    seanw@vsrx1> show version | display xml        
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <software-information>
            <host-name>vsrx1</host-name>
            <product-model>vsrx</product-model>
            <product-name>vsrx</product-name>
            <junos-version>15.1X49-D75.5</junos-version>
            <package-information>
                <name>junos</name>
                <comment>JUNOS Software Release [15.1X49-D75.5]</comment>
            </package-information>
        </software-information>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>    

In [1]:
from jnpr.junos import Device

vsrx1 = Device(host='vsrx1')
vsrx1.open()

Device(vsrx1)

PyEZ library does not contain a method for each Junos XML RPC and would require PyEZ to be tighly coupled to the Junos platform and version. With PyEZ there is no tight coupling and uses *metaprogramming* where each RPC gets the underscores replaced with hyphens and then wrapped in proper XML elements and sent over the NETCONF session to the device. PyEZ does not require formatting the RPC as XML, and it does not require directly interacting with NETCONF. The device instance’s *rpc* property can be used to execute an RPC. Each Junos XML RPC can be invoked as a method of the *rpc* property.

In [2]:
from lxml import etree

junos_version = vsrx1.rpc.get_software_information()
print etree.tostring(junos_version)

<software-information>
<host-name>vsrx1</host-name>
<product-model>vsrx</product-model>
<product-name>vsrx</product-name>
<junos-version>15.1X49-D75.5</junos-version>
<package-information>
<name>junos</name>
<comment>JUNOS Software Release [15.1X49-D75.5]</comment>
</package-information>
</software-information>



The **display_xml_rpc()** method provides the ability to ‘automatically’ find the appropriate RPC function to execute.

In [3]:
show_version_rpc = vsrx1.display_xml_rpc('show version', format='xml')
print etree.tostring(show_version_rpc)

<get-software-information>
</get-software-information>



It is also possible to pass CLI show command as an argument of the **cli()** method of the *rpc* property.

In [4]:
junos_version_txt = vsrx1.rpc.cli('show version', format='text')
print etree.tostring(junos_version_txt)

<output>
Hostname: vsrx1
Model: vsrx
Junos: 15.1X49-D75.5
JUNOS Software Release [15.1X49-D75.5]
</output>



### RPC Parameters
The PyEZ RPC mechanism allows the user to specify parameters without having to format them in XML. The parameters are simply specified as keyword arguments to the RPC methods.

    seanw@vsrx1> show route protocol bgp 2001:250:204::/48 active-path | display xml rpc 
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <rpc>
            <get-route-information>
                <destination>2001:250:204::/48</destination>
                <active-path/>
                <protocol>bgp</protocol>
            </get-route-information>
        </rpc>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>

In [5]:
bgp_route = vsrx1.rpc.get_route_information(protocol='bgp',
                                            destination='2001:250:204::/48',
                                            active_path=True)

### RPC Timeout
An RPC will timeout in 30 seconds by default if a complete response is not recieved (i.e. installing a Junos software package with the request-package-add RPC). You can set the device timeout which affects all RPCs.

In [6]:
vsrx1.timeout = 5

In [7]:
bgp_routes = vsrx1.rpc.get_route_information(dev_timeout=60,
                                            protocol='bgp')

Creating an algorithm for attempting to re-open a NETCONF connection can be done to re-execute a failed RPC a limited number of times. Importing the PyEZ exception module is required before specific PyEZ exceptions can be caught by an except statement.

In [8]:
from time import sleep

MAX_ATTEMPTS = 3
WAIT_BEFORE_RECONNECT = 10

for attempt in range(MAX_ATTEMPTS):
    try:
        routes = vsrx1.rpc.get_route_information()
    except jnpr.junos.exception.ConnectClosedError:
        sleep(WAIT_BEFORE_RECONNECT)
        try: vsrx1.open()
        except jnpr.junos.exception.ConnectError: pass
    else:
        # Success - no exception raied - break for loop
        break
else:
    # Max attempts exceeded and all failed - re-raise most recent exception
    raise
    
# Continue with rest or script if RPC succeeded ...

## Junos PyEZ RPC Responses
The default response to a NETCONF XML RPC is a string representing an XML document. PyEZ uses the *lxml* library to parse the response in a *lxml.etree.Element* object rooted at the first child element of the **`<rpc-reply>`** element. In the case below the first child element of the **`<rpc-reply>`** would be **`<system-user-information>`**.

    seanw@vsrx1> show system users | display xml    
    <rpc-reply xmlns:junos="http://xml.juniper.net/junos/15.1X49/junos">
        <system-users-information xmlns="http://xml.juniper.net/junos/15.1X49/junos">
            <uptime-information>
                <date-time junos:seconds="1486168021">12:27AM</date-time>
                <up-time junos:seconds="11565">3:12</up-time>
                <active-user-count junos:format="2 users">2</active-user-count>
                <load-average-1>0.04</load-average-1>
                <load-average-5>0.02</load-average-5>
                <load-average-15>0.00</load-average-15>
                <user-table>
                    <user-entry>
                        <user>seanw</user>
                        <tty>p0</tty>
                        <from>192.168.12.1</from>
                        <login-time junos:seconds="1486160956">Fri10PM</login-time>
                        <idle-time junos:seconds="0">-</idle-time>
                        <command>-cli (cli)</command>
                    </user-entry>
                    <user-entry>
                        <user>demo</user>
                        <tty>p1</tty>
                        <from>192.168.12.1</from>
                        <login-time junos:seconds="1486164395">Fri11PM</login-time>
                        <idle-time junos:seconds="282">4</idle-time>
                        <command>-cli (cli)</command>
                    </user-entry>
                </user-table>
            </uptime-information>
        </system-users-information>
        <cli>
            <banner></banner>
        </cli>
    </rpc-reply>

In [9]:
response = vsrx1.rpc.get_system_users_information()
response.tag

'system-users-information'

For debugging purposes, the **lxml.etree.dump()** function can be used to dump the XML text of the response (albeit without the pretty formatting of the Junos CLI).

    from lxml import etree
    etree.dump(response)

### Response Normalization
Response normalization is a PyEZ feature that actually alters the XML content returned from an RPC method. There are some Junos RPCs that return XML data where the values of certain XML elements are wrapped in newline or other whitespace characters. An example of this extra whitespace can be seen with the *get-system-users-information* RPC. Notice the text for the **`<up-time>`** element has a newline character before and after the value string. Response normalization is designed to address this situation. When response normalization is enabled, all whitespace characters at the beginning and end of each XML element’s value are removed. Response normalization is disabled by default (except when using tables and views).

In [10]:
response.findtext("uptime-information/up-time")

'\n7:42\n'

In [11]:
response = vsrx1.rpc.get_system_users_information(normalize=True)
response.findtext("uptime-information/up-time")

'7:42'

Normalization can be enabled for all RPCs on a device instance or in a NETCONF session by specifying the *normalize = True* argument to the **Device()** or **open()** calls, respectively.

    vsrx1 = Device(host = 'vsrx1', user = 'demo', password = 'Juniper', normalize = True)

### lxml Elements
**lxml** APIs can be used to select information from an RPC response. The text content of the first XML element matching an XPath expression can be retrieved with the **findtext()** method. The argument to the **findtext()** method is an XPath relative to the *response* element. Because *response* represents the **`<system-users-information>`** element, the **uptime-information/up-time** XPath matches the **`<up-time>`** tag in the *response*.

In [12]:
response.findtext("uptime-information/up-time")

'7:42'

The **`<up-time>`** element also contains a *seconds* attribute that provides the system’s uptime in an easier to parse number of seconds since the system booted. The value of this attribute can be accessed by chaining the **find()** method and the *attrib* attribute. While the **findtext()** method returns a string, the **find()** method returns an *lxml.etree.Element* object. The XML attributes of that *lxml.etree.Element* object can then be accessed using the *attrib* dictionary. The *attrib* dictionary is keyed using the XML attribute name.

In [13]:
response.find("uptime-information/up-time").attrib["seconds"]

'27777'

In the response variable, there is one **`<user-entry>`** XML element for each user currently logged into the device. Each **`<user-entry>`** element contains a **`<user>`** element with the user’s username. The **findall()** method returns a list of *lxml.etree.Element* objects matching an XPath. In this example, **findall()** is used to select the **`<user>`** element within every **`<user-entry>`** element.

In [14]:
users = response.findall("uptime-information/user-table/user-entry/user")
for user in users:
    print user.text.strip()

seanw
demo


The next example combines the **findtext()** method with an XPath expression that selects the first matching XML element that has a specific matching child element.

In [15]:
print response.findtext("uptime-information/user-table/user-entry[tty='p0']/user")

seanw


In [16]:
XPATH = "uptime-information/user-table/user-entry[user='demo']/idle-time"
response.find(XPATH).attrib['seconds']

'473'

### jxmlease 
The jxmlease library can also be used to directly parse an *lxml.etree.Element* object by passing an *lxml.etree.Element* object to an instance of the *jxmlease.EtreeParser* class.

In [17]:
import jxmlease

parser = jxmlease.EtreeParser()
new_response = parser(response)
new_response.prettyprint(depth=3)

{'system-users-information': {'uptime-information': {'active-user-count': u'2',
                                                     'date-time': u'5:12AM',
                                                     'load-average-1': u'0.32',
                                                     'load-average-15': u'0.03',
                                                     'load-average-5': u'0.10',
                                                     'up-time': u'7:42',
                                                     'user-table': {...}}}}


The response is actually a *jxmlease.XMLDictNode* object, but behaves much like an ordered dictionary.

In [18]:
type(new_response)

jxmlease.dictnode.XMLDictNode

Access to any level of the *jxmlease.XMLDictNode* object can be done by specifying a chain of dictionary keys. The tag of each XML element is used as a dictionary key, and begins with the tag of the RPC response’s root element.

In [19]:
print new_response['system-users-information']['uptime-information']['up-time']

7:42


For XML elements that have attributes, you can use the object’s **get_xml_attr()** method to retrieve the attribute’s value. The **get_xml_attr()** also allows a default value to be returned if the XML attribute name does not exist.

In [20]:
up_time = new_response['system-users-information']['uptime-information']['up-time']
print up_time.get_xml_attr('seconds')

27777


In [21]:
print up_time.get_xml_attr('foo', 0)

0


In [22]:
vsrx1.close()